# Setting Up Environment

In [12]:
import pandas as pd
import re

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Calculate Bets

In [5]:
# Instantiating webdriver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.actionnetwork.com/ufc/odds')

# Getting odds table and formatting
html = driver.page_source
tables = pd.read_html(html)
odds = tables[0]
odds = odds.iloc[::2]
odds.reset_index(drop = True, inplace = True)

[WDM] - Downloading: 100%|█████████████████| 8.79M/8.79M [00:01<00:00, 7.36MB/s]
/var/folders/xz/nv9k4jpn7l9dtk4l7yqbl8640000gn/T/ipykernel_8481/3244408514.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [75]:
# Iterating through to get each fighter's odds

odds_df = pd.DataFrame(columns = ['Fighter_1', 'Fighter_2', 'Fighter_1_Odds', 'Fighter_2_Odds'])
fighter_2_regex = r'^[A-Za-z]+\s[A-Za-z]+'
fighter_1_regex = r'[A-Za-z]+\s[A-Za-z]+(?=[A-Za-z]*\.)'
for index, row in odds.iterrows():
    # Getting fighter names
    try:
        fighter_2 = re.findall(fighter_2_regex, row.Scheduled)[0]
        fighter_2 = fighter_2[:-1]
        fighter_2_replace = fighter_2.split()[1]
        fighter_1 = re.findall(fighter_1_regex, row.Scheduled)[1]
        fighter_1 = fighter_1.replace(fighter_2_replace, "")[:-1]
    except:
        flag_regex = r'[^\x00-\x7F]'
        names_string = re.sub(flag_regex, '', row.Scheduled)
        fighter_2 = re.findall(fighter_2_regex, names_string)[0]
        fighter_2 = fighter_2[:-1]
        fighter_2_replace = fighter_2.split()[1]
        fighter_1 = re.findall(fighter_1_regex, names_string)[1]
        fighter_1 = fighter_1.replace(fighter_2_replace, "")[:-1]
    # Getting fighter odds
    ml_string = row['Unnamed: 3']
    if len(ml_string) == 8:
        ml_fighter_2 = ml_string[:4]
        ml_fighter_1 = ml_string[-4:]
    elif len(ml_string) == 9:
        if (ml_string[4] == '+') | (ml_string[4]=='-'):
            ml_fighter_2 = ml_string[:4]
            ml_fighter_1 = ml_string[-5:]
        else:
            ml_fighter_2 = ml_string[:5]
            ml_fighter_1 = ml_string[-4:]
    elif len(ml_string) == 10:
            ml_fighter_2 = ml_string[:5]
            ml_fighter_1 = ml_string[-5:]
    else:
        continue
    try:
        ml_fighter_2 = float(ml_fighter_2)
    except:
        continue
    try:
        ml_fighter_1 = float(ml_fighter_1)
    except:
        continue
    # Adding data to odds df
    new_data = [fighter_1, fighter_2, ml_fighter_1, ml_fighter_2]
    new_df = pd.DataFrame([new_data])
    new_df.columns = odds_df.columns
    odds_df = pd.concat([odds_df, new_df], ignore_index = True)

print(odds_df)

          Fighter_1             Fighter_2  Fighter_1_Odds  Fighter_2_Odds
0      Karine Silva    Priscila Cachoeira          -196.0           152.0
1  Francis Marshall         William Gomis          -225.0           172.0
2    Mohammed Usman           Junior Tafa          -104.0          -120.0
3        Karol Rosa          Norma Dumont          -108.0          -118.0
4        Rani Yahya        Montel Jackson           430.0          -670.0
5    Jeremiah Wells  Matthew Semelsberger          -115.0          -111.0
6    Iasmin Lucindo         Brogan Walker          -330.0           240.0
7       Bobby Green          Jared Gordon          -265.0           200.0
8      Brad Tavares           Bruno Silva          -180.0           140.0
9  Sergei Pavlovich        Curtis Blaydes           134.0          -172.0


In [44]:
odds

,Scheduled,Open,Best Odds,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,Brady HiestandB. HiestandDanaa BatgerelD. Batg...,+135-177,+110-132,N/AN/A,N/AN/A,N/AN/A,N/AN/A,+102-132,+110-143,+110-143,N/AN/A,N/AN/A,N/AN/A
1,🇧🇷Priscila CachoeiraP. CachoeiraKarine SilvaK....,+136-170,+160-180,+152-196,N/AN/A,+160-190,N/AN/A,+155-200,+155-190,+150-180,N/AN/A,N/AN/A,N/AN/A
2,William GomisW. GomisFrancis MarshallF. Marshall,+170-200,+180-210,+172-225,N/AN/A,+180-210,N/AN/A,+160-215,+170-210,N/AN/A,N/AN/A,+180-220,N/AN/A
3,Junior TafaJ. TafaMohammed UsmanM. Usman,-250+210,-110-104,-120-104,N/AN/A,-110-110,N/AN/A,-114-114,-115-105,-115-105,N/AN/A,-110-110,N/AN/A
4,🇧🇷Norma DumontN. Dumont🇧🇷Karol RosaK. Rosa,+120-140,-115+100,-118-108,N/AN/A,-115-105,N/AN/A,-127-103,N/AN/A,-115-105,N/AN/A,-120+100,N/AN/A
5,🇺🇸Montel JacksonM. Jackson🇧🇷Rani YahyaR. Yahya,-560+430,-560+430,-670+430,N/AN/A,-560+430,N/AN/A,N/AN/A,-600+425,-600+425,N/AN/A,-600+430,N/AN/A
6,🇺🇸Christos GiagosC. Giagos🇺🇸Rick GlennR. Glenn,+195-230,+150-167,N/AN/A,N/AN/A,+150-175,N/AN/A,+130-167,+135-167,+140-175,N/AN/A,N/AN/A,N/AN/A
7,Matthew SemelsbergerM. SemelsbergerJeremiah We...,+130-150,-110-105,-111-115,N/AN/A,-110-110,N/AN/A,-114-114,-118-105,-118-105,N/AN/A,-110-110,N/AN/A
8,Brogan WalkerB. WalkerIasmin LucindoI. Lucindo,+230-275,+255-300,+240-330,N/AN/A,+255-305,N/AN/A,+225-305,+240-300,+240-300,N/AN/A,+240-300,N/AN/A
9,Jared GordonJ. GordonBobby GreenB. Green,+240-285,+215-250,+200-265,N/AN/A,+210-250,N/AN/A,+200-275,+215-270,+215-270,N/AN/A,+210-260,N/AN/A
